# Reading, cleaning and scaling

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

def fullRead(pathToTable, sep, anthro = False):

  df_renamed = pd.read_csv(pathToTable, sep = sep, encoding = "latin_1")


  # reading and merging    
  
  if anthro == True:
    df_anthro = pd.read_csv("data/chronicAnthropometricCardiovascularData.csv", sep=";", decimal=",")
    df_renamed = df_renamed.merge(df_anthro)

    # separating by time moment and renaming

    df_renamed["Weight"] = ""
    df_renamed["BMI"] = ""
    df_renamed["Fat"] = ""
    df_renamed["CVRI"] = ""
    df_renamed["Bpmin"] = ""
    df_renamed["Bpmax"] = ""
    df_renamed["Frec"] = ""

    for i in range(len(df_renamed)):

        if df_renamed.loc[i]["Time"] == "Initial":
            df_renamed.loc[i,"Weight"] = df_renamed.loc[i]["Peso inicial"]
            df_renamed.loc[i,"BMI"] = df_renamed.loc[i]["IMC Inicial"]
            df_renamed.loc[i,"Fat"] = df_renamed.loc[i]["Grasa inicial"]
            df_renamed.loc[i,"CVRI"] = df_renamed.loc[i]["IRCV inicial"] 
            df_renamed.loc[i,"Bpmin"] = df_renamed.loc[i]["Bpmin inicial"] 
            df_renamed.loc[i,"Bpmax"] = df_renamed.loc[i]["Bpmax inicial"] 
            df_renamed.loc[i,"Frec"] = df_renamed.loc[i]["Frec inicial"] 
                
        if df_renamed.loc[i]["Time"] == "Final":
        
            df_renamed.loc[i,"Weight"] = df_renamed.loc[i]["Peso final"]
            df_renamed.loc[i,"BMI"] = df_renamed.loc[i]["IMC Final"]
            df_renamed.loc[i,"Fat"] = df_renamed.loc[i]["Grasa final"]
            df_renamed.loc[i,"CVRI"] = df_renamed.loc[i]["IRCV Final"] 
            df_renamed.loc[i,"Bpmin"] = df_renamed.loc[i]["Bpmin final"] 
            df_renamed.loc[i,"Bpmax"] = df_renamed.loc[i]["Bpmax final"] 
            df_renamed.loc[i,"Frec"] = df_renamed.loc[i]["Frec final"] 
        
    df_renamed.drop(columns = ["Peso inicial", "Peso final", "Delta Peso", "Talla", "IMC Inicial", "IMC Final", "Delta IMC", "Grasa inicial", "Grasa final", "Delta Grasa", "IRCV Final", "IRCV inicial", "Bpmin final", "Bpmin inicial", "Bpmax final", "Bpmax inicial", "Frec final", "Frec inicial",], inplace=True )
  
  df_renamed.drop(columns = ["Unnamed: 0", "grouping"], inplace=True )
  df_renamed.fillna(0, inplace=True)
  return df_renamed

def scaling(df_read):
   
   scaler = preprocessing.MinMaxScaler()
   numCols = df_read.select_dtypes(include=np.number).drop("numVol",1).columns
   df_read[numCols] = scaler.fit_transform(df_read[numCols])
   return df_read

df = scaling(fullRead("data/plasmAnt_ord.csv",  sep = ",", anthro= True))

C:\Users\dres2\AppData\Local\Temp\ipykernel_14256\4052692268.py:56: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  numCols = df_read.select_dtypes(include=np.number).drop("numVol",1).columns


# Split and encoding

In [2]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
 
enc = OrdinalEncoder()
enc.fit(df[["Sweetener", "Sex", "Time"]])
df[["Sweetener", "Sex", "Time"]] = enc.transform(df[["Sweetener", "Sex",  "Time"]])

# Initial = 1, Final = 0

X, y = df[df["Time"] == 1].drop(["numVol", "Time"], axis=1), df[df["Time"] == 0].drop(['Time','numVol', 'Sweetener', 'Time', 'Sex', 'Weight','BMI', 'Fat', 'CVRI', 'Bpmin', 'Bpmax', 'Frec'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_full, y_full = df[df["Time"] == 1].drop(["numVol", "Time"], axis=1), df[df["Time"] == 0].drop(['Time','numVol', 'Sweetener', 'Time'], axis = 1)
X_fulltrain, X_fulltest, y_fulltrain, y_fulltest = train_test_split(X_full, y_full, test_size=0.3, random_state=42)

#X_test.to_csv("X_met_test_urineAnt.csv", index=False)
#X_fulltest.to_csv("X_full_test_urineAnt.csv",index=False)

RF - metabolics

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
import pickle

param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# define model
model = RandomForestRegressor()

grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(X_train, y_train)
best_grid = grid_search.best_estimator_
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(best_grid, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = np.absolute(n_scores)
# summarize performance

#filename = 'rf_met_plasmAnt.pkl'
#with open(filename, 'wb') as file:
#    pickle.dump(grid_search.best_estimator_, file)

print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))



Fitting 3 folds for each of 288 candidates, totalling 864 fits
MAE: 0.119 (0.010)


# RF FULL

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# define model
model = RandomForestRegressor()

grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(X_fulltrain, y_fulltrain)
best_grid = grid_search.best_estimator_
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(best_grid, X_full, y_full, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = np.absolute(n_scores)

#filename = 'rf_full_plasmAnt.pkl'
#with open(filename, 'wb') as file:
#    pickle.dump(grid_search.best_estimator_, file)

# summarize performance
print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

Fitting 3 folds for each of 288 candidates, totalling 864 fits
MAE: 0.116 (0.008)


# mlp for multi-output regression


In [ ]:

# Full parameters gridsearch
# DONT EXECUTE
'''
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection import GridSearchCV

from scikeras.wrappers import KerasRegressor
import tensorflow as tf


# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]

param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer, optimizer__learning_rate=learn_rate, optimizer__momentum=momentum,
                  model__init_mode=init_mode, model__activation=activation, model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

# get the model
def get_model(n_inputs, n_outputs, dropout_rate, weight_constraint, activation="relu", init_mode='uniform'):
    model_nn = Sequential()
    model_nn.add(Dense(64, input_shape=(X.shape[1],),activation=activation, kernel_initializer=init_mode, kernel_constraint=MaxNorm(weight_constraint)))
    model_nn.add(Dropout(dropout_rate))
    model_nn.add(Dense(32, activation=activation, kernel_initializer=init_mode))
    model_nn.add(Dropout(dropout_rate))
    model_nn.add(Dense(y.shape[1], activation='linear', kernel_initializer=init_mode))
    return model_nn
 
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
 results = list()
 n_inputs, n_outputs = X.shape[1], y.shape[1]
 # define evaluation procedure
 cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# define modeld
 model_nn = KerasRegressor(model = get_model) #(n_inputs, n_outputs, activation="relu", init_mode='uniform'))
 # fit model
 grid = GridSearchCV(estimator=model_nn, param_grid=param_grid, n_jobs=-1, cv=3)
 grid_result = grid.fit(X_train, y_train) # evaluate model on test set
 mae = model_nn.evaluate(X_test, y_test)
 # store result
 print('>%.3f' % mae)
 results.append(mae)
 return results
 
# evaluate model
results = evaluate_model(X, y)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))
'''

MLP only metabolics

In [3]:
# mlp for multi-output regression

from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
import pickle

tf.get_logger().setLevel('ERROR')


# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

epochs = [10, 50, 100]
batch_size = [10, 20, 40, 60, 80, 100]

param_grid = dict(batch_size=batch_size, epochs=epochs)

# get the model
def get_model(n_inputs, n_outputs):
    model_nn = Sequential()
    model_nn.add(Dense(64, input_shape=(X.shape[1],),activation="relu"))
    model_nn.add(Dropout(0.5))
    model_nn.add(Dense(32, activation="relu"))
    model_nn.add(Dropout(0.5))
    model_nn.add(Dense(y.shape[1], activation='linear'))
    model_nn.compile(loss='mae', optimizer=tf.keras.optimizers.Adam())

    return model_nn
 
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
 results = list()
 n_inputs, n_outputs = X.shape[1], y.shape[1]
 # define evaluation procedure
 cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# define modeld
 model_nn = KerasRegressor(model = get_model(n_inputs, n_outputs), optimizer=tf.keras.optimizers.Adam(), verbose=0)
 # fit model
 grid = GridSearchCV(estimator=model_nn, param_grid=param_grid, n_jobs=-1, cv=3, verbose=0)
 grid_result = grid.fit(X_train, y_train) # evaluate model on test set
 # summarize results
 best_grid = grid_result.best_estimator_
# define the evaluation procedure
 cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
 n_scores = cross_val_score(best_grid, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
 n_scores = np.absolute(n_scores)

# store the model
# filename = 'mlp_met_plasmAnt.pkl'
# with open(filename, 'wb') as file:
#    pickle.dump(grid_result.best_estimator_, file)

# summarize performance
 print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

# evaluate model
evaluate_model(X, y)


MAE: 0.112 (0.013)


MLP full

In [4]:
# mlp for multi-output regression

from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
import pickle
from sklearn.model_selection import cross_val_score

tf.get_logger().setLevel('ERROR')


# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

epochs = [10, 50, 100]
batch_size = [10, 20, 40, 60, 80, 100]

param_grid = dict(batch_size=batch_size, epochs=epochs)

# get the model
def get_model(n_inputs, n_outputs):
    model_nn = Sequential()
    model_nn.add(Dense(64, input_shape=(X.shape[1],),activation="relu"))
    model_nn.add(Dropout(0.5))
    model_nn.add(Dense(32, activation="relu"))
    model_nn.add(Dropout(0.5))
    model_nn.add(Dense(y_full.shape[1], activation='linear'))
    model_nn.compile(loss='mae', optimizer=tf.keras.optimizers.Adam())

    return model_nn
 
def evaluate_model(X, y):
 results = list()
 n_inputs, n_outputs = X.shape[1], y.shape[1]
 # define evaluation procedure
 cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# define modeld
 model_nn = KerasRegressor(model = get_model(n_inputs, n_outputs), optimizer=tf.keras.optimizers.Adam(), verbose=0)
 # fit model
 grid = GridSearchCV(estimator=model_nn, param_grid=param_grid, n_jobs=-1, cv=3, verbose=0)
 grid_result = grid.fit(X_train, y_train) # evaluate model on test set
 # summarize results
 best_grid = grid_result.best_estimator_
# define the evaluation procedure
 cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
 n_scores = cross_val_score(best_grid, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
 n_scores = np.absolute(n_scores)

# store the model
# filename = 'mlp_full_plasmAnt.pkl'
# with open(filename, 'wb') as file:
#    pickle.dump(grid_result.best_estimator_, file)

# summarize performance
 print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

# evaluate model
evaluate_model(X_full, y_full)


MAE: 0.110 (0.010)
